<a href="https://colab.research.google.com/github/dellielo/test/blob/master/test_lib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install https://github.com/dellielo/kahelo/archive/v.0.1.0.zip
!pip install -U segmentation-models
import cv2
import kahelo



     |████████████████████████████████| 1.6MB 532kB/s 
  Created wheel for kahelo: filename=kahelo-1.0.1-cp36-none-any.whl size=21343 sha256=80d36780afbd0b2caef21172f1e7ac6db2a72960ffa98ab5d730a3461bc24027
  Stored in directory: /tmp/pip-ephem-wheel-cache-hoeprbjj/wheels/c5/5e/f3/819b1139e741f3150b1e6acb46c9ec7bf52f340ca988077917
Successfully built kahelo
3.4.3


AttributeError: ignored

In [12]:
print(cv2.__version__)
print(dir(kahelo))

3.4.3
['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [0]:
!pip install -U albumentations>=0.3.0 --user 
!pip install -U --pre segmentation-models --user

